In [ ]:
#@title Cell 1 - WAN2GP Setup Introduction + Enhanced Platform Detection (Bug Fixed)

import sys
import os
import subprocess
from IPython.display import display, HTML, Markdown

def jupyter_lab_title(title):
    if 'google.colab' not in sys.modules:
        display(Markdown(f"## {title}"))

jupyter_lab_title("WAN2GP Setup Introduction + Enhanced Platform Detection")

def detect_platform():
    """Enhanced platform detection with comprehensive indicators"""

    # Lightning AI detection - multiple indicators for reliability
    lightning_indicators = [
        'lightning' in str(sys.executable).lower(),
        'teamspace-studios' in os.getcwd(),
        'LIGHTNING_CLOUD_SPACE_HOST' in os.environ,
        'LIGHTNING_CLOUD_SPACE_ID' in os.environ,
        'commands/python' in str(sys.executable),
        '/home/zeus/miniconda3/envs/cloudspace' in str(sys.executable),
        os.path.exists('/teamspace'),
        os.path.exists('/commands')
    ]

    # Google Colab detection
    colab_indicators = [
        'google.colab' in sys.modules,
        '/content' in os.getcwd()
    ]

    # Vast.AI detection
    vast_indicators = [
        'VAST_CONTAINERLABEL' in os.environ,
        '/workspace' in os.getcwd(),
        'vast' in os.environ.get('HOSTNAME', '').lower()
    ]

    if any(lightning_indicators):
        return "Lightning AI"
    elif any(colab_indicators):
        return "Google Colab"
    elif any(vast_indicators):
        return "Vast.AI/Generic"
    else:
        return "Vast.AI/Generic"

def verify_virtual_environment(platform, venv_path):
    """Verify virtual environment exists and is functional"""
    if platform == "Lightning AI":
        return True, "pip", "python"  # Lightning AI uses system packages

    # Check if virtual environment directory exists
    if not os.path.exists(venv_path):
        print(f"Creating virtual environment at {venv_path}")
        try:
            subprocess.run([sys.executable, '-m', 'venv', venv_path], check=True)
        except subprocess.CalledProcessError as e:
            print(f"Failed to create virtual environment: {e}")
            return False, "pip", "python"

    # Check if pip and python binaries exist
    if os.name == 'nt':  # Windows
        pip_path = os.path.join(venv_path, 'Scripts', 'pip.exe')
        python_path = os.path.join(venv_path, 'Scripts', 'python.exe')
    else:  # Unix-like
        pip_path = os.path.join(venv_path, 'bin', 'pip')
        python_path = os.path.join(venv_path, 'bin', 'python')

    if not os.path.exists(pip_path) or not os.path.exists(python_path):
        print(f"Virtual environment binaries missing. Using system Python.")
        return False, "pip", "python"

    return True, pip_path, python_path

def get_platform_commands(platform):
    """Get platform-specific pip and python commands with verification"""
    if platform == "Lightning AI":
        return "pip", "python", False  # pip_cmd, python_cmd, use_venv
    elif platform == "Google Colab":
        venv_exists, pip_cmd, python_cmd = verify_virtual_environment(platform, ".venv")
        return pip_cmd, python_cmd, venv_exists
    else:  # Vast.AI/Generic
        venv_exists, pip_cmd, python_cmd = verify_virtual_environment(platform, ".venv")
        return pip_cmd, python_cmd, venv_exists

# Global variable initialization with error handling
def initialize_global_platform_vars():
    """Initialize global platform variables with comprehensive error handling"""
    try:
        current_platform = detect_platform()
        pip_cmd, python_cmd, use_venv = get_platform_commands(current_platform)

        # Verify the commands actually work
        try:
            subprocess.run([python_cmd, '--version'], capture_output=True, check=True)
            print(f"✅ Python command verified: {python_cmd}")
        except (subprocess.CalledProcessError, FileNotFoundError):
            print(f"⚠️ Python command failed, falling back to system python")
            python_cmd = "python"
            use_venv = False

        try:
            subprocess.run([pip_cmd, '--version'], capture_output=True, check=True)
            print(f"✅ Pip command verified: {pip_cmd}")
        except (subprocess.CalledProcessError, FileNotFoundError):
            print(f"⚠️ Pip command failed, falling back to system pip")
            pip_cmd = "pip"
            use_venv = False

        return current_platform, pip_cmd, python_cmd, use_venv
    except Exception as e:
        print(f"❌ Platform detection failed: {e}")
        print("Using fallback configuration")
        return "Generic", "pip", "python", False

# Initialize global variables
current_platform, pip_cmd, python_cmd, use_venv = initialize_global_platform_vars()

# Make variables globally accessible
globals()['current_platform'] = current_platform
globals()['pip_cmd'] = pip_cmd
globals()['python_cmd'] = python_cmd
globals()['use_venv'] = use_venv

# Display platform information
display(HTML(f"""
<div style="font-family: Arial, sans-serif; background: linear-gradient(135deg, #667eea 0%, #764ba2 100%); color: white; padding: 20px; border-radius: 10px; text-align: center; margin: 10px 0; box-shadow: 0 4px 15px rgba(0,0,0,0.2);">
    <h1>🚀 Wan2GP Full Setup Notebook (Bug Fixed)</h1>
    <p>A cross-platform Jupyter notebook (Colab / Lightning AI / Vast.ai) with comprehensive bug fixes and error handling.</p>
</div>
<div style="background-color: #f8f9fa; padding: 15px; border-radius: 8px; margin: 10px 0; border: 1px solid #e9ecef;">
    <h3>🔧 Platform Detection Results (Fixed)</h3>
    <p><strong>Platform:</strong> <span style="color: #28a745;">{current_platform}</span></p>
    <p><strong>Virtual Environment:</strong> <span style="color: #28a745;">{"Yes" if use_venv else "No (System)"}</span></p>
    <p><strong>Pip:</strong> <code>{pip_cmd}</code></p>
    <p><strong>Python:</strong> <code>{python_cmd}</code></p>
</div>
"""))

print("\n" + "="*50)
print("🔍 Platform Detection Debug Info:")
print(f"- Detected Platform: {current_platform}")
print(f"- Python Executable: {sys.executable}")
print(f"- Current Working Directory: {os.getcwd()}")
print(f"- Virtual Environment Usage: {use_venv}")
print("✅ Global variables initialized and verified")
print("="*50)
